In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler,OrdinalEncoder

In [2]:
t1 = pd.read_csv('T1.csv')
t2 = pd.read_csv('Wind Data.csv')
s1 = pd.read_csv('Plant_1_Weather_Sensor_Data.csv')
s2 = pd.read_csv('Plant_2_Weather_Sensor_Data.csv')


In [3]:
s = pd.concat([s1, s2], ignore_index=True)


In [4]:
g1 = pd.read_csv("Plant_1_Generation_Data.csv")
g2 = pd.read_csv("Plant_2_Generation_Data.csv")

g = pd.concat([g1,g2], ignore_index=True)

In [5]:
t1.head()

,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645904,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


In [6]:
t2.head()

,Time,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month
0,2017-01-01 00:00:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
1,2017-01-01 00:15:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
2,2017-01-01 00:30:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
3,2017-01-01 00:45:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
4,2017-01-01 01:00:00,0,0.0,1.7,1020,100,5.2,0.0,0.0,100,0,0,450,0.0,4,1,1


In [7]:
s.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0


In [8]:
g.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
1,15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
2,15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
3,15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
4,15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0


In [9]:
def check_null_unique(df):
    null_data = []
    nunique_vals = 0
    null_per = 0
    unique_vals = []
    for i in df.columns:
        null_val = df[i].isna().sum()
        null_per = (np.round(((null_val)/len(df)),3))*100
        nunique_vals = int(df[i].nunique())
        unique_vals = list(df[i].unique())
        null_data.append([i,null_per,nunique_vals,unique_vals])


    return pd.DataFrame(null_data, columns=['Column', 'Null_Percentage','nunique','unique_vals']) if null_data else pd.DataFrame(columns=['Column', 'Null_Percentage','nunique','unique_vals'])

In [10]:
check_null_unique(t1)

,Column,Null_Percentage,nunique,unique_vals
0,Date/Time,0.0,50530,"[01 01 2018 00:00, 01 01 2018 00:10, 01 01 201..."
1,LV ActivePower (kW),0.0,38764,"[380.0477905, 453.7691956, 306.3765869, 419.64..."
2,Wind Speed (m/s),0.0,50305,"[5.31133604, 5.672166824, 5.216036797, 5.65967..."
3,Theoretical_Power_Curve (KWh),0.0,37157,"[416.3289078, 519.9175111, 390.9000158, 516.12..."
4,Wind Direction (°),0.0,50251,"[259.9949036, 268.6411133, 272.5647888, 271.25..."


In [11]:
check_null_unique(t2)

,Column,Null_Percentage,nunique,unique_vals
0,Time,0.0,196776,"[2017-01-01 00:00:00, 2017-01-01 00:15:00, 201..."
1,Energy delta[Wh],0.0,4556,"[0, 5, 33, 44, 61, 65, 83, 69, 98, 138, 161, 1..."
2,GHI,0.0,2277,"[0.0, 0.2, 2.7, 6.4, 10.6, 6.0, 2.8, 3.1, 3.5,..."
3,temp,0.0,503,"[1.6, 1.7, 1.9, 2.0, 2.5, 2.6, 2.8, 2.9, 3.5, ..."
4,pressure,0.0,71,"[1021, 1020, 1019, 1018, 1017, 1016, 1015, 101..."
5,humidity,0.0,79,"[100, 99, 97, 93, 91, 90, 89, 88, 94, 96, 98, ..."
6,wind_speed,0.0,136,"[4.9, 5.2, 5.5, 5.7, 5.6, 6.0, 6.1, 6.2, 5.8, ..."
7,rain_1h,0.0,311,"[0.0, 0.19, 0.26, 0.33, 0.51, 0.16, 0.1, 0.53,..."
8,snow_1h,0.0,129,"[0.0, 0.62, 0.53, 0.34, 0.12, 0.23, 0.47, 0.52..."
9,clouds_all,0.0,101,"[100, 98, 94, 79, 80, 48, 27, 23, 9, 4, 5, 3, ..."


In [12]:
check_null_unique(s)

,Column,Null_Percentage,nunique,unique_vals
0,DATE_TIME,0.0,3262,"[2020-05-15 00:00:00, 2020-05-15 00:15:00, 202..."
1,PLANT_ID,0.0,2,"[4135001, 4136001]"
2,SOURCE_KEY,0.0,2,"[HmiyD2TTLFNqkNe, iq8k7ZNt4Mwm3w0]"
3,AMBIENT_TEMPERATURE,0.0,6441,"[25.184316133333333, 25.08458866666667, 24.935..."
4,MODULE_TEMPERATURE,0.0,6441,"[22.8575074, 22.761667866666663, 22.5923055333..."
5,IRRADIATION,0.0,3620,"[0.0, 0.0008627212413333, 0.0058869571866666, ..."


In [13]:
check_null_unique(g)

,Column,Null_Percentage,nunique,unique_vals
0,DATE_TIME,0.0,6417,"[15-05-2020 00:00, 15-05-2020 00:15, 15-05-202..."
1,PLANT_ID,0.0,2,"[4135001, 4136001]"
2,SOURCE_KEY,0.0,44,"[1BY6WEcLGh8j5v7, 1IF53ai7Xc0U56Y, 3PZuoBAID5W..."
3,DC_POWER,0.0,63607,"[0.0, 37.14285714, 53.5, 58.0, 58.42857143, 54..."
4,AC_POWER,0.0,62915,"[0.0, 3.585714286, 5.1625, 5.585714286, 5.6285..."
5,DAILY_YIELD,0.0,59307,"[0.0, 2.625, 3.0, 3.5, 3.142857143, 3.375, 2.8..."
6,TOTAL_YIELD,0.0,70382,"[6259559.0, 6183645.0, 6987759.0, 7602960.0, 7..."


In [14]:
print('t1',t1.shape)
print('t2',t2.shape)
print('s',s.shape)
print('g',g.shape)

t1 (50530, 5)
t2 (196776, 17)
s (6441, 6)
g (136476, 7)


### **🔹 Recommended Approach** 🚀   
            
Since we aim to **forecast solar energy generation**, we should ensure that our data is clean, structured, and ready for modeling. Here's my step-by-step p   lan:           
           
---
            
### **1️⃣ Data Cleaning & Preprocessing**                      
✅ **Fix Date Format Issues**                    
- Dataset 1 has `DATE_TIME` in `YYYY-MM-DD HH:MM:SS` format.                 
- Dataset 2 has `DATE_TIME` in `DD-MM-YYYY HH:MM`.               
- Convert both to a consistent **datetime format**.          
           
✅ **Check for Missing Values**              
- Since `Null_Percentage = 0.0`, we don't need imputation, but we should **verify inconsistencies**.       
           
✅ **Ensure Data Consistency**                               
- Some features like `SOURCE_KEY` appear in both datasets but have **different unique values**.                          
- We should confirm whether they refer to **same sensors** or **different sources**.           
                   
---            
            
### **2️⃣ Merging Both Datasets**               
✅ **Merge on `DATE_TIME`, `PLANT_ID`, and `SOURCE_KEY`**               
- This will combine **environmental factors (Dataset 1)** with **power generation data (Dataset 2)**.            
            
---             
            
### **3️⃣ Feature Engineering & Selection**               
✅ **Create New Features**                
- **Solar Efficiency** = `AC_POWER / DC_POWER` → Helps analyze panel efficiency.             
- **Temperature Impact** = `MODULE_TEMPERATURE - AMBIENT_TEMPERATURE` → Determines heat loss effects.               
- **Time-Based Features**: Extract **hour, day, month** from `DATE_TIME` for **time-series modeling**.              
             
✅ **Drop Irrelevant Features**              
- If `SOURCE_KEY` is redundant (same for both datasets), we might remove it.  
                    
---                 
              
### **4️⃣ Exploratory Data Analysis (EDA) & Correlation Check**             
✅ **Check Feature Importance**                
- Find **correlations** between `IRRADIATION`, `TEMPERATURE`, and `POWER OUTPUT`.                 
- Use **heatmaps, scatter plots, and time-series analysis**.             
                 
---             
             
### **5️⃣ Model Selection & Forecasting Approach**                
🔹 **ML Models (Short-Term Forecasting)**               
- **Random Forest, XGBoost, LSTM (for time-series prediction)**.                
         
🔹 **Deep Learning (Long-Term Forecasting)**                      
- LSTM or Transformer-based models for **multi-step forecasting**.                          
---                        
               
### **🔹 Next Steps: Start with Data Cleaning?**                                   
Would you like me to **clean & merge the datasets now?** 🛠️

# Working on the solar dataset

Step 1: Data Cleaning & Preprocessing 🛠️        
✅ 1.1 Fix Date Format Issues        
Your datasets have different formats for the DATE_TIME column:       
          
Dataset 1: YYYY-MM-DD HH:MM:SS        
Dataset 2: DD-MM-YYYY HH:MM           
🔹 Solution: Convert both to a consistent format (YYYY-MM-DD HH:MM:SS).         

✅ 1.2 Check for Missing Values                                    
Your provided summary shows 0% missing values, which is great! But let's double-check for hidden inconsistencies (e.g., NaN values in different encodings).             
                                     
✅ 1.3 Data Type Consistency
Ensure numeric columns (AMBIENT_TEMPERATURE, DC_POWER, etc.) are floats or integers.                              
Ensure DATE_TIME is converted to a datetime object for easy merging.                      

In [15]:
# 1️⃣ Fix DATE_TIME Format
# Convert Dataset 1 (YYYY-MM-DD HH:MM:SS format is correct)
s["DATE_TIME"] = pd.to_datetime(s["DATE_TIME"], format="%Y-%m-%d %H:%M")


ValueError: unconverted data remains when parsing with format "%Y-%m-%d %H:%M": ":00", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
g1.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
1,15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
2,15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
3,15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
4,15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0


In [ ]:
g2.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,2020-05-15 00:00:00,4136001,4UPUqMRk7TRMgml,0.0,0.0,9425.000000,2.429011e+06
1,2020-05-15 00:00:00,4136001,81aHJ1q11NBPMrL,0.0,0.0,0.000000,1.215279e+09
2,2020-05-15 00:00:00,4136001,9kRcWv60rDACzjR,0.0,0.0,3075.333333,2.247720e+09
3,2020-05-15 00:00:00,4136001,Et9kgGMDl729KT4,0.0,0.0,269.933333,1.704250e+06
4,2020-05-15 00:00:00,4136001,IQ2d7wF4YD8zU1Q,0.0,0.0,3177.000000,1.994153e+07


In [ ]:
# Convert Dataset 2 (DD-MM-YYYY HH:MM to YYYY-MM-DD HH:MM:SS)
g1["DATE_TIME"] = pd.to_datetime(g1["DATE_TIME"], format="%d-%m-%Y %H:%M")
g2["DATE_TIME"] = pd.to_datetime(g2["DATE_TIME"], format="%Y-%m-%d %H:%M:%S")


In [ ]:
g1['DATE_TIME'] = pd.to_datetime(g1['DATE_TIME'])
g2['DATE_TIME'] = pd.to_datetime(g2['DATE_TIME'])

In [ ]:
g1.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,2020-05-15,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
1,2020-05-15,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
2,2020-05-15,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
3,2020-05-15,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
4,2020-05-15,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0


In [ ]:
g2.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,2020-05-15,4136001,4UPUqMRk7TRMgml,0.0,0.0,9425.000000,2.429011e+06
1,2020-05-15,4136001,81aHJ1q11NBPMrL,0.0,0.0,0.000000,1.215279e+09
2,2020-05-15,4136001,9kRcWv60rDACzjR,0.0,0.0,3075.333333,2.247720e+09
3,2020-05-15,4136001,Et9kgGMDl729KT4,0.0,0.0,269.933333,1.704250e+06
4,2020-05-15,4136001,IQ2d7wF4YD8zU1Q,0.0,0.0,3177.000000,1.994153e+07


In [ ]:
g1.sample(20)

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
42595,2020-06-05 12:45:00,4135001,rGa61gmuvPhdLxV,5427.857143,532.100000,3964.285714,7269313.286
4974,2020-05-17 12:15:00,4135001,ZnxXDlPa8U1GXgE,10550.750000,1030.475000,4172.375000,6539324.375
47686,2020-06-07 23:45:00,4135001,WRmjgnKYAwPKWDb,0.000000,0.000000,7062.125000,7204682.000
42527,2020-06-05 12:00:00,4135001,ih0vzX44oOqAx2f,4302.714286,422.342857,3485.000000,6340504.000
11474,2020-05-20 19:30:00,4135001,ZnxXDlPa8U1GXgE,0.000000,0.000000,8909.000000,6564045.000
37389,2020-06-03 00:45:00,4135001,bvBOhCH3iADSZry,0.000000,0.000000,0.000000,6445487.000
30697,2020-05-30 20:15:00,4135001,zVJPv84UY57bAof,0.000000,0.000000,7290.000000,7235977.000
136,2020-05-15 01:30:00,4135001,adLQvlD726eNBSB,0.000000,0.000000,0.000000,6271355.000
25889,2020-05-28 05:15:00,4135001,7JYdWkrLSPkdwr4,0.000000,0.000000,0.000000,7698550.000
53296,2020-06-10 15:30:00,4135001,WRmjgnKYAwPKWDb,4733.125000,464.400000,5660.250000,7226638.250


In [ ]:
g2.sample(20)

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
37266,2020-06-03 13:45:00,4136001,oZZkBaNadn6DNKz,550.140000,531.383333,3688.166667,1.708211e+09
55948,2020-06-12 10:15:00,4136001,xMbIugepa2P7lBB,244.500000,239.746667,1025.466667,1.068662e+08
54845,2020-06-11 21:45:00,4136001,q49J1IKaHRwDQnt,0.000000,0.000000,3814.000000,4.867550e+05
16378,2020-05-23 13:15:00,4136001,oZ35aAeoifZaQzV,1240.386667,1209.393333,6165.400000,1.660034e+09
29934,2020-05-31 02:15:00,4136001,Qf4GUc1pJu5T6c6,0.000000,0.000000,0.000000,8.385435e+08
18435,2020-05-24 17:45:00,4136001,xoJJ8DcxJEcupym,126.973333,123.513333,9034.533333,2.091958e+08
33163,2020-06-01 15:00:00,4136001,LYwnQax7tkwH5Cb,56.313333,54.480000,3781.733333,1.795051e+09
59327,2020-06-14 00:45:00,4136001,Quc1TzYxW2pYoWX,0.000000,0.000000,5477.000000,3.296334e+08
18490,2020-05-24 18:45:00,4136001,4UPUqMRk7TRMgml,0.000000,0.000000,4635.000000,2.497601e+06
63645,2020-06-16 01:45:00,4136001,q49J1IKaHRwDQnt,0.000000,0.000000,6060.000000,5.113250e+05


concatenate with updated g1

In [ ]:
g = pd.concat([g1,g2], ignore_index=True)

In [ ]:
g.sample(20)

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
98459,2020-05-30 23:15:00,4136001,xoJJ8DcxJEcupym,0.000000,0.000000,5088.000000,2.092334e+08
69584,2020-05-15 09:00:00,4136001,mqwcsP2rE7J0TFp,871.920000,853.140000,1286.400000,5.935813e+08
79345,2020-05-20 02:30:00,4136001,9kRcWv60rDACzjR,0.000000,0.000000,0.000000,2.247757e+09
28603,2020-05-29 20:30:00,4135001,uHbuxQJl8lW7ozc,0.000000,0.000000,0.000000,7.151932e+06
72110,2020-05-16 14:00:00,4136001,Qf4GUc1pJu5T6c6,819.360000,801.786667,6415.333333,8.384342e+08
40458,2020-06-04 11:45:00,4135001,wCURE6d3bPkepu2,10582.250000,1033.537500,3341.750000,6.934118e+06
89147,2020-05-25 20:45:00,4136001,PeE6FRyGXUgsRhN,0.000000,0.000000,8737.000000,1.348422e+09
127294,2020-06-13 15:30:00,4136001,mqwcsP2rE7J0TFp,697.973333,682.720000,4866.333333,5.937919e+08
73185,2020-05-17 02:15:00,4136001,Mx2yZCDsyf6DPfv,0.000000,0.000000,0.000000,2.467253e+06
131945,2020-06-15 20:30:00,4136001,81aHJ1q11NBPMrL,0.000000,0.000000,6307.000000,1.215476e+09


In [ ]:
g['DC_POWER'].isna().sum()

np.int64(0)

In [ ]:
g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136476 entries, 0 to 136475
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATE_TIME    136476 non-null  datetime64[ns]
 1   PLANT_ID     136476 non-null  int64         
 2   SOURCE_KEY   136476 non-null  object        
 3   DC_POWER     136476 non-null  float64       
 4   AC_POWER     136476 non-null  float64       
 5   DAILY_YIELD  136476 non-null  float64       
 6   TOTAL_YIELD  136476 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 7.3+ MB


In [ ]:
s['DATE_TIME'] = pd.to_datetime(s['DATE_TIME'])

In [ ]:
s.sample(20)

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
4081,2020-05-24 09:15:00,4136001,iq8k7ZNt4Mwm3w0,30.751167,48.793326,0.671354
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.000000
3993,2020-05-23 11:15:00,4136001,iq8k7ZNt4Mwm3w0,34.156459,56.555210,0.948992
4432,2020-05-28 01:00:00,4136001,iq8k7ZNt4Mwm3w0,26.525676,25.465288,0.000000
3629,2020-05-19 16:15:00,4136001,iq8k7ZNt4Mwm3w0,22.933313,21.121700,0.023003
3225,2020-05-15 10:45:00,4136001,iq8k7ZNt4Mwm3w0,33.192470,50.249560,0.943353
3619,2020-05-19 13:30:00,4136001,iq8k7ZNt4Mwm3w0,33.952788,53.399307,0.839524
6126,2020-06-14 17:15:00,4136001,iq8k7ZNt4Mwm3w0,26.899515,30.419657,0.143601
398,2020-05-19 06:15:00,4135001,HmiyD2TTLFNqkNe,23.344480,22.342457,0.009697
1282,2020-05-29 00:30:00,4135001,HmiyD2TTLFNqkNe,21.008919,20.190776,0.000000


In [ ]:
s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6441 entries, 0 to 6440
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATE_TIME            6441 non-null   datetime64[ns]
 1   PLANT_ID             6441 non-null   int64         
 2   SOURCE_KEY           6441 non-null   object        
 3   AMBIENT_TEMPERATURE  6441 non-null   float64       
 4   MODULE_TEMPERATURE   6441 non-null   float64       
 5   IRRADIATION          6441 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 302.1+ KB


In [ ]:
# Check for Missing Values
missing_env = s.isnull().sum()
missing_power = g.isnull().sum()
print("Missing Values in Environmental Data:\n", missing_env)
print("Missing Values in Power Data:\n", missing_power)


Missing Values in Environmental Data:
 DATE_TIME              0
PLANT_ID               0
SOURCE_KEY             0
AMBIENT_TEMPERATURE    0
MODULE_TEMPERATURE     0
IRRADIATION            0
dtype: int64
Missing Values in Power Data:
 DATE_TIME      0
PLANT_ID       0
SOURCE_KEY     0
DC_POWER       0
AC_POWER       0
DAILY_YIELD    0
TOTAL_YIELD    0
dtype: int64


In [ ]:
# Ensure Correct Data Types
# Convert numerical columns to float (if not already)
numeric_cols_env = ["AMBIENT_TEMPERATURE", "MODULE_TEMPERATURE", "IRRADIATION"]
numeric_cols_power = ["DC_POWER", "AC_POWER", "DAILY_YIELD", "TOTAL_YIELD"]


In [ ]:
# Confirm data types
print("\nEnvironmental Data Types:\n", s.dtypes)
print("\nPower Data Types:\n", g.dtypes)



Environmental Data Types:
 DATE_TIME              datetime64[ns]
PLANT_ID                        int64
SOURCE_KEY                     object
AMBIENT_TEMPERATURE           float64
MODULE_TEMPERATURE            float64
IRRADIATION                   float64
dtype: object

Power Data Types:
 DATE_TIME      datetime64[ns]
PLANT_ID                int64
SOURCE_KEY             object
DC_POWER              float64
AC_POWER              float64
DAILY_YIELD           float64
TOTAL_YIELD           float64
dtype: object


In [ ]:
# Check for Duplicate Rows
duplicates_env = s.duplicated().sum()
duplicates_power = g.duplicated().sum()
print("\nDuplicate Rows in Environmental Data:", duplicates_env)
print("Duplicate Rows in Power Data:", duplicates_power)


Duplicate Rows in Environmental Data: 0
Duplicate Rows in Power Data: 0


In [ ]:
# Save cleaned datasets (optional)
s.to_csv("cleaned_environmental_data.csv", index=False)
g.to_csv("cleaned_power_data.csv", index=False)

In [ ]:
# Merge Datasets on DATE_TIME
merged_df = pd.merge(s, g, on=["DATE_TIME", "PLANT_ID", "SOURCE_KEY"], how="inner")
print("\nMerged Dataset Sample:\n", merged_df.head())


Merged Dataset Sample:
 Empty DataFrame
Columns: [DATE_TIME, PLANT_ID, SOURCE_KEY, AMBIENT_TEMPERATURE, MODULE_TEMPERATURE, IRRADIATION, DC_POWER, AC_POWER, DAILY_YIELD, TOTAL_YIELD]
Index: []


In [ ]:
merged_df.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD


In [ ]:
merged_df.shape

(0, 10)

In [ ]:
print(s["DATE_TIME"].head(), g["DATE_TIME"].head())  # Check formats  
print(set(s["PLANT_ID"]).difference(set(g["PLANT_ID"])))  # Check plant ID mismatches  
print(set(s["SOURCE_KEY"]).difference(set(g["SOURCE_KEY"])))  # Check source key mismatches  


0   2020-05-15 00:00:00
1   2020-05-15 00:15:00
2   2020-05-15 00:30:00
3   2020-05-15 00:45:00
4   2020-05-15 01:00:00
Name: DATE_TIME, dtype: datetime64[ns] 0   2020-05-15
1   2020-05-15
2   2020-05-15
3   2020-05-15
4   2020-05-15
Name: DATE_TIME, dtype: datetime64[ns]
set()
{'iq8k7ZNt4Mwm3w0', 'HmiyD2TTLFNqkNe'}


In [ ]:
g["DATE_TIME"] = pd.to_datetime(g["DATE_TIME"], format="%Y-%m-%d %H:%M")


In [ ]:
print(s["DATE_TIME"].head(), g["DATE_TIME"].head())  # Check formats  


0   2020-05-15 00:00:00
1   2020-05-15 00:15:00
2   2020-05-15 00:30:00
3   2020-05-15 00:45:00
4   2020-05-15 01:00:00
Name: DATE_TIME, dtype: datetime64[ns] 0   2020-05-15
1   2020-05-15
2   2020-05-15
3   2020-05-15
4   2020-05-15
Name: DATE_TIME, dtype: datetime64[ns]


In [ ]:
s["DATE_TIME"] = s["DATE_TIME"].dt.floor("5min")
g["DATE_TIME"] = g["DATE_TIME"].dt.floor("5min")

In [ ]:
s.sample(15)

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
4271,2020-05-26 08:45:00,4136001,iq8k7ZNt4Mwm3w0,29.250903,41.693755,0.568577
4073,2020-05-24 07:15:00,4136001,iq8k7ZNt4Mwm3w0,26.727255,29.838664,0.209922
4082,2020-05-24 09:30:00,4136001,iq8k7ZNt4Mwm3w0,31.119519,50.532769,0.720494
3614,2020-05-19 12:15:00,4136001,iq8k7ZNt4Mwm3w0,32.560873,52.558259,0.922836
6009,2020-06-13 12:00:00,4136001,iq8k7ZNt4Mwm3w0,31.011079,46.759196,0.728177
1197,2020-05-28 03:15:00,4135001,HmiyD2TTLFNqkNe,22.443715,20.575374,0.000000
1065,2020-05-26 18:15:00,4135001,HmiyD2TTLFNqkNe,29.093020,28.168933,0.024506
367,2020-05-18 22:30:00,4135001,HmiyD2TTLFNqkNe,22.850214,20.732054,0.000000
4147,2020-05-25 01:45:00,4136001,iq8k7ZNt4Mwm3w0,25.668540,23.945862,0.000000
5534,2020-06-08 13:15:00,4136001,iq8k7ZNt4Mwm3w0,33.378355,50.701859,0.618060


In [ ]:
g.sample(15)

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
100726,2020-06-01 01:15:00,4136001,4UPUqMRk7TRMgml,0.000000,0.000000,6692.000000,2.552559e+06
123009,2020-06-11 14:45:00,4136001,vOuJvMaM2sgwLmb,253.606667,248.733333,3147.733333,2.392179e+06
66924,2020-06-17 02:15:00,4135001,sjndEbLyjtCKgGv,0.000000,0.000000,0.000000,7.255794e+06
87100,2020-05-24 16:15:00,4136001,oZ35aAeoifZaQzV,593.773333,581.826667,4513.933333,1.660042e+09
55471,2020-06-11 16:15:00,4135001,7JYdWkrLSPkdwr4,4398.285714,431.771429,5168.285714,7.806259e+06
71189,2020-05-16 03:30:00,4136001,WcxssY2VbP4hApt,0.000000,0.000000,0.000000,1.817049e+08
11819,2020-05-21 08:15:00,4135001,1IF53ai7Xc0U56Y,5972.500000,585.237500,532.000000,6.225959e+06
116668,2020-06-08 14:45:00,4136001,mqwcsP2rE7J0TFp,990.180000,965.973333,6648.933333,5.541814e+08
27659,2020-05-29 09:00:00,4135001,ih0vzX44oOqAx2f,7151.750000,700.137500,794.125000,6.287964e+06
33747,2020-06-01 07:00:00,4135001,z9Y9gH1T5YWrNuG,2084.428571,204.328571,97.285714,7.133994e+06


In [ ]:
print(s["DATE_TIME"].head(5), g["DATE_TIME"].head(5))  # Check formats  
print(set(s["PLANT_ID"]).difference(set(g["PLANT_ID"])))  # Check plant ID mismatches  
print(set(s["SOURCE_KEY"]).difference(set(g["SOURCE_KEY"])))  # Check source key mismatches  


0   2020-05-15 00:00:00
1   2020-05-15 00:15:00
2   2020-05-15 00:30:00
3   2020-05-15 00:45:00
4   2020-05-15 01:00:00
Name: DATE_TIME, dtype: datetime64[ns] 0   2020-05-15
1   2020-05-15
2   2020-05-15
3   2020-05-15
4   2020-05-15
Name: DATE_TIME, dtype: datetime64[ns]
set()
{'iq8k7ZNt4Mwm3w0', 'HmiyD2TTLFNqkNe'}


In [ ]:
# Save cleaned datasets (optional)
s.to_csv("cleaned_environmental_data.csv", index=False)
g.to_csv("cleaned_power_data.csv", index=False)

In [ ]:
print(set(s["SOURCE_KEY"]).difference(set(g["SOURCE_KEY"])))  # Check source key mismatches  


{'iq8k7ZNt4Mwm3w0', 'HmiyD2TTLFNqkNe'}


In [ ]:
g['SOURCE_KEY'].unique()

array(['1BY6WEcLGh8j5v7', '1IF53ai7Xc0U56Y', '3PZuoBAID5Wc2HD',
       '7JYdWkrLSPkdwr4', 'McdE0feGgRqW7Ca', 'VHMLBKoKgIrUVDU',
       'WRmjgnKYAwPKWDb', 'ZnxXDlPa8U1GXgE', 'ZoEaEvLYb1n2sOq',
       'adLQvlD726eNBSB', 'bvBOhCH3iADSZry', 'iCRJl6heRkivqQ3',
       'ih0vzX44oOqAx2f', 'pkci93gMrogZuBj', 'rGa61gmuvPhdLxV',
       'sjndEbLyjtCKgGv', 'uHbuxQJl8lW7ozc', 'wCURE6d3bPkepu2',
       'z9Y9gH1T5YWrNuG', 'zBIq5rxdHJRwDNY', 'zVJPv84UY57bAof',
       'YxYtjZvoooNbGkE', '4UPUqMRk7TRMgml', '81aHJ1q11NBPMrL',
       '9kRcWv60rDACzjR', 'Et9kgGMDl729KT4', 'IQ2d7wF4YD8zU1Q',
       'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'Mx2yZCDsyf6DPfv',
       'NgDl19wMapZy17u', 'PeE6FRyGXUgsRhN', 'Qf4GUc1pJu5T6c6',
       'Quc1TzYxW2pYoWX', 'V94E5Ben1TlhnDV', 'WcxssY2VbP4hApt',
       'mqwcsP2rE7J0TFp', 'oZ35aAeoifZaQzV', 'oZZkBaNadn6DNKz',
       'q49J1IKaHRwDQnt', 'rrq4fwE8jgrTyWY', 'vOuJvMaM2sgwLmb',
       'xMbIugepa2P7lBB', 'xoJJ8DcxJEcupym'], dtype=object)

# Dropped Source Id

In [16]:
snew = s.drop(columns=['SOURCE_KEY'])
gnew = g.drop(columns=['SOURCE_KEY'])

In [17]:
snew.head()

,DATE_TIME,PLANT_ID,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,4135001,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,4135001,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,4135001,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,4135001,24.621525,22.165423,0.0


In [18]:
gnew.head()

,DATE_TIME,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,15-05-2020 00:00,4135001,0.0,0.0,0.0,6259559.0
1,15-05-2020 00:00,4135001,0.0,0.0,0.0,6183645.0
2,15-05-2020 00:00,4135001,0.0,0.0,0.0,6987759.0
3,15-05-2020 00:00,4135001,0.0,0.0,0.0,7602960.0
4,15-05-2020 00:00,4135001,0.0,0.0,0.0,7158964.0


In [19]:
snew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6441 entries, 0 to 6440
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DATE_TIME            6441 non-null   object 
 1   PLANT_ID             6441 non-null   int64  
 2   AMBIENT_TEMPERATURE  6441 non-null   float64
 3   MODULE_TEMPERATURE   6441 non-null   float64
 4   IRRADIATION          6441 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 251.7+ KB


In [20]:
merged_df = pd.merge(snew, gnew, left_on=['DATE_TIME', 'PLANT_ID'], right_on=['DATE_TIME', 'PLANT_ID'], how='inner')


In [21]:
merged_df

,DATE_TIME,PLANT_ID,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,2020-05-15 00:00:00,4136001,27.004764,25.060789,0.0,0.0,0.0,9425.000000,2.429011e+06
1,2020-05-15 00:00:00,4136001,27.004764,25.060789,0.0,0.0,0.0,0.000000,1.215279e+09
2,2020-05-15 00:00:00,4136001,27.004764,25.060789,0.0,0.0,0.0,3075.333333,2.247720e+09
3,2020-05-15 00:00:00,4136001,27.004764,25.060789,0.0,0.0,0.0,269.933333,1.704250e+06
4,2020-05-15 00:00:00,4136001,27.004764,25.060789,0.0,0.0,0.0,3177.000000,1.994153e+07
...,...,...,...,...,...,...,...,...,...
67693,2020-06-17 23:45:00,4136001,23.202871,22.535908,0.0,0.0,0.0,4157.000000,5.207580e+05
67694,2020-06-17 23:45:00,4136001,23.202871,22.535908,0.0,0.0,0.0,3931.000000,1.211314e+08
67695,2020-06-17 23:45:00,4136001,23.202871,22.535908,0.0,0.0,0.0,4322.000000,2.427691e+06
67696,2020-06-17 23:45:00,4136001,23.202871,22.535908,0.0,0.0,0.0,4218.000000,1.068964e+08


In [22]:
check_null_unique(merged_df)

,Column,Null_Percentage,nunique,unique_vals
0,DATE_TIME,0.0,3259,"[2020-05-15 00:00:00, 2020-05-15 00:15:00, 202..."
1,PLANT_ID,0.0,1,[4136001]
2,AMBIENT_TEMPERATURE,0.0,3259,"[27.004763700000005, 26.880811433333328, 26.68..."
3,MODULE_TEMPERATURE,0.0,3259,"[25.0607889, 24.421868833333328, 24.4272903103..."
4,IRRADIATION,0.0,1863,"[0.0, 0.002838054505, 0.0129619781466666, 0.02..."
5,DC_POWER,0.0,30825,"[0.0, 15.406666666666665, 14.773333333333332, ..."
6,AC_POWER,0.0,30783,"[0.0, 14.86, 14.246666666666668, 14.84, 14.44,..."
7,DAILY_YIELD,0.0,30490,"[9425.0, 0.0, 3075.3333333333344, 269.93333333..."
8,TOTAL_YIELD,0.0,33115,"[2429011.0, 1215278736.0, 2247719577.0, 170425..."


In [23]:
merged_df.drop(columns=['DC_POWER'],inplace= True)

In [25]:
merged_df.drop(columns=['TOTAL_YIELD'],inplace= True)

In [27]:
merged_df.drop(columns=['PLANT_ID'],inplace= True)

In [28]:
check_null_unique(merged_df)

,Column,Null_Percentage,nunique,unique_vals
0,DATE_TIME,0.0,3259,"[2020-05-15 00:00:00, 2020-05-15 00:15:00, 202..."
1,AMBIENT_TEMPERATURE,0.0,3259,"[27.004763700000005, 26.880811433333328, 26.68..."
2,MODULE_TEMPERATURE,0.0,3259,"[25.0607889, 24.421868833333328, 24.4272903103..."
3,IRRADIATION,0.0,1863,"[0.0, 0.002838054505, 0.0129619781466666, 0.02..."
4,AC_POWER,0.0,30783,"[0.0, 14.86, 14.246666666666668, 14.84, 14.44,..."
5,DAILY_YIELD,0.0,30490,"[9425.0, 0.0, 3075.3333333333344, 269.93333333..."


In [29]:
merged_df.to_csv('complete_solar_data.csv',index=False)

# Now we will work on wind dataset